In [1]:
import pickle
# import plotly.express as px
import numpy as np
import datetime

In [7]:
import os
os.path.j

'2023-11-26 13:00:27'

In [6]:
str(datetime.datetime.now())

'2023-11-26 13:00:06.988104'

In [8]:
d = pickle.load(open('C:/Users/ivand/Desktop/ses/test_2023-11-26-13-33-20/render_params.pickle','rb'))

In [9]:
d

{'resolution_percentage': (100,),
 'resolution_x': (1920,),
 'resolution_y': (1080,),
 'pixel_aspect_x': (1.0,),
 'pixel_aspect_y': (1.0,),
 'engine': 'BLENDER_EEVEE'}

In [ ]:
class Camera():
    def __init__(self, **kwargs):
        self.params = kwargs
        self.

    def get_sensor_size(sensor_fit, sensor_x, sensor_y):
        if sensor_fit == 'VERTICAL':
            return sensor_y
        return sensor_x

    # BKE_camera_sensor_fit
    def get_sensor_fit(sensor_fit, size_x, size_y):
        if sensor_fit == 'AUTO':
            if size_x >= size_y:
                return 'HORIZONTAL'
            else:
                return 'VERTICAL'
        return sensor_fit

    # Build intrinsic camera parameters from Blender camera data
    #
    # See notes on this in 
    # blender.stackexchange.com/questions/15102/what-is-blenders-camera-projection-matrix-model
    # as well as
    # https://blender.stackexchange.com/a/120063/3581
    def get_calibration_matrix_K_from_blender(camd):
        if camd.type != 'PERSP':
            raise ValueError('Non-perspective cameras not supported')
        f_in_mm = camd.lens
        scale = scene.render.resolution_percentage / 100
        resolution_x_in_px = scale * scene.render.resolution_x
        resolution_y_in_px = scale * scene.render.resolution_y
        sensor_size_in_mm = get_sensor_size(camd.sensor_fit, camd.sensor_width, camd.sensor_height)
        sensor_fit = get_sensor_fit(
            camd.sensor_fit,
            scene.render.pixel_aspect_x * resolution_x_in_px,
            scene.render.pixel_aspect_y * resolution_y_in_px
        )
        pixel_aspect_ratio = scene.render.pixel_aspect_y / scene.render.pixel_aspect_x
        if sensor_fit == 'HORIZONTAL':
            view_fac_in_px = resolution_x_in_px
        else:
            view_fac_in_px = pixel_aspect_ratio * resolution_y_in_px
        pixel_size_mm_per_px = sensor_size_in_mm / f_in_mm / view_fac_in_px
        s_u = 1 / pixel_size_mm_per_px
        s_v = 1 / pixel_size_mm_per_px / pixel_aspect_ratio

        # Parameters of intrinsic calibration matrix K
        u_0 = resolution_x_in_px / 2 - camd.shift_x * view_fac_in_px
        v_0 = resolution_y_in_px / 2 + camd.shift_y * view_fac_in_px / pixel_aspect_ratio
        skew = 0 # only use rectangular pixels

        K = np.array(
            [(s_u, skew, u_0),
            (   0,  s_v, v_0),
            (   0,    0,   1)])
        return K

In [6]:
for o in d:
    if o['type']=='CAMERA':
        break

In [7]:
o

{'type': 'CAMERA',
 'name': 'Camera',
 'location': array([ 7.35889149, -6.92579079,  4.95830917]),
 'scale': array([1., 1., 1.]),
 'rotation_euler': array([1.10931897, 0.        , 0.81492817]),
 'lens': 50.0,
 'lens_unit': 'MILLIMETERS',
 'angle_x': 0.6911112070083618,
 'angle_y': 0.4710899591445923,
 'sensor_fit': 'AUTO',
 'sensor_height': 24.0,
 'sensor_width': 36.0,
 'imname': 'C:/Users/ivand/Desktop/ses/Camera.png'}

In [ ]:
import bpy
import pickle
import mathutils
import numpy as np


import bpy
from mathutils import Matrix, Vector
import numpy as np

# From https://blender.stackexchange.com/a/120063/116769
# copy get_sensor_size(), get_sensor_fit(), get_calibration_matrix_K_from_blender()

#---------------------------------------------------------------
# 3x4 P matrix from Blender camera
#---------------------------------------------------------------

# BKE_camera_sensor_size


# Our contribution below
def get_3x4_RT_matrix_from_blender(obj):
  isCamera = (obj.type == 'CAMERA')
  R_BlenderView_to_OpenCVView = np.diag([1 if isCamera else -1,-1,-1])

  location, rotation = obj.matrix_world.decompose()[:2]
  R_BlenderView = rotation.to_matrix().transposed()

  T_BlenderView = -1.0 * R_BlenderView @ location

  R_OpenCV = R_BlenderView_to_OpenCVView @ R_BlenderView
  T_OpenCV = R_BlenderView_to_OpenCVView @ T_BlenderView
  
  RT_OpenCV = Matrix(np.column_stack((R_OpenCV, T_OpenCV)))
  return RT_OpenCV, R_OpenCV, T_OpenCV

def get_3x4_P_matrix_from_blender(cam):
  K = get_calibration_matrix_K_from_blender(cam.data)
  RT = get_3x4_RT_matrix_from_blender(cam)[0]
  return K @ RT

def export_camera_to_opencv(cam_object):
  P = get_3x4_P_matrix_from_blender(cam_object)

  nP = np.matrix(P)
  return nP